In [1]:
import copy
import numpy as np
import os
import cv2 as cv
from skimage.feature import hog
from tqdm import tqdm

In [2]:
def IOU(bbox_a, bbox_b):
    x_a = max(bbox_a[0], bbox_b[0])
    y_a = max(bbox_a[1], bbox_b[1])
    x_b = min(bbox_a[2], bbox_b[2])
    y_b = min(bbox_a[3], bbox_b[3])

    inter_area = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

    box_a_area = (bbox_a[2] - bbox_a[0] + 1) * (bbox_a[3] - bbox_a[1] + 1)
    box_b_area = (bbox_b[2] - bbox_b[0] + 1) * (bbox_b[3] - bbox_b[1] + 1)

    iou = inter_area / float(box_a_area + box_b_area - inter_area)

    return iou


In [3]:
def prelucrare_adnotari(fisier_adnotari):
    dict_adnotari = {}
    with open(fisier_adnotari, "r") as txt_file:
        content = txt_file.readlines()

    for row in content:
        linie = row.strip().split()
        image_name = linie[0]
        coordinates = list(map(int, linie[1:5]))
        nume = linie[5]

        if image_name not in dict_adnotari:
            dict_adnotari[image_name] = {}

        if nume not in dict_adnotari[image_name]:
            dict_adnotari[image_name][nume] = []

        dict_adnotari[image_name][nume].append(coordinates)

    return dict_adnotari


In [4]:
dim_fereastra_patrata = 96
latime_fereastra_dreptunghi = 96
inaltime_fereastra_dreptunghi = 64
dim_celuala_hog_patrat = 12
dim_celula_hog_dreptunghi = 12
orient = 12

In [5]:
def descriptori_negativi(fisier_imagini,dictionar_adnotari):
    files=os.listdir(fisier_imagini)
    # os.makedirs("./exemple_img_neg",exist_ok=True)
    nr_imagini_salvate = 1000
    nr_random_latime = 10
    nr_max_incercari = 150
    descriptori_patrat = []
    descriptori_dreptunghi = []
    descriptori_dreptunghi_inalt = []
    patch_salvate = []
    
    for file in tqdm(files):
        # print(file)
        patch=None
        img = cv.imread(fisier_imagini + file)
        img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        img = np.array(img, dtype=np.uint8)
        
        #generam patchuri random square
        for _ in range(nr_random_latime):
            for _ in range(nr_max_incercari):
                latime_aleasa = np.random.randint(20,270)
                inaltime_aleasa = latime_aleasa 
                # print(f"latime_aleasa: {latime_aleasa}")
                # print(inaltime_aleasa)
                x_min = np.random.randint(0,img.shape[1]-latime_aleasa)
                y_min = np.random.randint(0,img.shape[0]-inaltime_aleasa)
                x_max = x_min + latime_aleasa
                y_max = y_min +latime_aleasa
                bbox1 = [x_min,y_min,x_max,y_max]
                valid = True
                keys = dictionar_adnotari[file].keys()
                for key in keys:
                    if not valid:
                        break
                    for bbox2 in dictionar_adnotari[file][key]:
                        if IOU(bbox1,bbox2) > 0.3:
                            valid = False
                            break
                for p in patch_salvate:
                    if IOU(bbox1, p) > 0.3:
                        valid = False
                        break
                if valid:
                    #prelucram patchul
                    patch_salvate.append(bbox1)
                    patch = img[y_min:y_max, x_min:x_max]
                    # if nr_imagini_salvate>0:
                    #     cv.imwrite(f"./exemple_img_neg/patrat_{nr_imagini_salvate}.jpg", patch)
                    #     nr_imagini_salvate = nr_imagini_salvate - 1
                    patch = cv.resize(patch, (dim_fereastra_patrata, dim_fereastra_patrata))
                    descriptor = hog(patch, pixels_per_cell=(dim_celuala_hog_patrat, dim_celuala_hog_patrat), cells_per_block=(2, 2), feature_vector=True, block_norm='L2-Hys',orientations = orient )
                    descriptori_patrat.append(descriptor)
                    break
        patch_salvate=[] 
        #generam patchuri random dreptunghi
        for _ in range(nr_random_latime):
            for _ in range(nr_max_incercari):
                latime_aleasa = np.random.randint(20,270)
                inaltime_aleasa = (2*latime_aleasa)//3
                x_min = np.random.randint(0,img.shape[1]-latime_aleasa)
                y_min = np.random.randint(0,img.shape[0]-inaltime_aleasa)
                x_max = x_min + latime_aleasa
                y_max = y_min + inaltime_aleasa
                bbox1 = [x_min,y_min,x_max,y_max]
                valid = True
                keys = dictionar_adnotari[file].keys()
                for key in keys:
                    if not valid:
                        break
                    for bbox2 in dictionar_adnotari[file][key]:
                        if IOU(bbox1,bbox2) > 0.3:
                            valid = False
                            break
                for p in patch_salvate:
                    if IOU(bbox1, p) > 0.3:
                        valid = False
                        break
                if valid:
                    #prelucram patchul
                    patch_salvate.append(bbox1)
                    patch = img[y_min:y_max, x_min:x_max]
                    # if nr_imagini_salvate>0:
                    #     cv.imwrite(f"./exemple_img_neg/dreptunghi_{nr_imagini_salvate}.jpg", patch)
                    #     nr_imagini_salvate = nr_imagini_salvate -1
                    patch = cv.resize(patch, (latime_fereastra_dreptunghi, inaltime_fereastra_dreptunghi))
                    descriptor = hog(patch, pixels_per_cell=(dim_celula_hog_dreptunghi, dim_celula_hog_dreptunghi), cells_per_block=(2, 2), feature_vector=True, block_norm='L2-Hys',orientations = orient)
                    descriptori_dreptunghi.append(descriptor)
                    break
        patch_salvate=[]
        #generam patchuri random dreptunghi inalt
        for _ in range(nr_random_latime):
            for _ in range(nr_max_incercari):
                latime_aleasa = np.random.randint(20,230)
                inaltime_aleasa = (3*latime_aleasa)//2
                x_min = np.random.randint(0,img.shape[1]-latime_aleasa)
                y_min = np.random.randint(0,img.shape[0]-inaltime_aleasa)
                x_max = x_min + latime_aleasa
                y_max = y_min + inaltime_aleasa
                bbox1 = [x_min,y_min,x_max,y_max]
                valid = True
                keys = dictionar_adnotari[file].keys()
                for key in keys:
                    if not valid:
                        break
                    for bbox2 in dictionar_adnotari[file][key]:
                        if IOU(bbox1,bbox2) > 0.3:
                            valid = False
                            break
                for p in patch_salvate:
                    if IOU(bbox1, p) > 0.3:
                        valid = False
                        break
                if valid:
                    #prelucram patchul
                    patch_salvate.append(bbox1)
                    patch = img[y_min:y_max, x_min:x_max]
                    # if nr_imagini_salvate>0:
                    #     cv.imwrite(f"./exemple_img_neg/dreptunghi_inalt_{nr_imagini_salvate}.jpg", patch)
                    #     nr_imagini_salvate = nr_imagini_salvate -1
                    patch = cv.resize(patch, (inaltime_fereastra_dreptunghi,latime_fereastra_dreptunghi))
                    descriptor = hog(patch, pixels_per_cell=(dim_celula_hog_dreptunghi, dim_celula_hog_dreptunghi), cells_per_block=(2, 2), feature_vector=True,block_norm='L2-Hys',orientations = orient)
                    descriptori_dreptunghi_inalt.append(descriptor)
                    break
    return descriptori_patrat, descriptori_dreptunghi, descriptori_dreptunghi_inalt



In [6]:
dict_dad = prelucrare_adnotari('./antrenare/dad_annotations.txt')
dict_deedee = prelucrare_adnotari('./antrenare/deedee_annotations.txt')
dict_dexter = prelucrare_adnotari('./antrenare/dexter_annotations.txt')
dict_mom = prelucrare_adnotari('./antrenare/mom_annotations.txt')
os.makedirs('./descriptori_negativi',exist_ok=True)
os.makedirs('./descriptori_negativi/patrat',exist_ok=True)
os.makedirs('./descriptori_negativi/dreptunghi',exist_ok=True)
os.makedirs('./descriptori_negativi/dreptunghi_inalt',exist_ok=True)

patrat, drept, drept_lung = descriptori_negativi('./antrenare/dad/',dict_dad)
descr_patrat = copy.deepcopy(patrat) 
descr_drept = copy.deepcopy(drept)
descr_lung = copy.deepcopy(drept_lung)

patrat, drept, drept_lung = descriptori_negativi('./antrenare/deedee/',dict_deedee)
descr_patrat = np.concatenate([descr_patrat,patrat])
descr_drept = np.concatenate([descr_drept,drept])
descr_lung = np.concatenate([descr_lung,drept_lung])

patrat, drept, drept_lung = descriptori_negativi('./antrenare/dexter/',dict_dexter)
descr_patrat = np.concatenate([descr_patrat,patrat])
descr_drept = np.concatenate([descr_drept,drept])
descr_lung = np.concatenate([descr_lung,drept_lung])

patrat, drept, drept_lung = descriptori_negativi('./antrenare/mom/',dict_mom)
descr_patrat = np.concatenate([descr_patrat,patrat])
descr_drept = np.concatenate([descr_drept,drept])
descr_lung = np.concatenate([descr_lung,drept_lung])

np.save(f'./descriptori_negativi/patrat/descr_neg',descr_patrat)

np.save(f'./descriptori_negativi/dreptunghi/descr_neg',descr_drept)

np.save(f'./descriptori_negativi/dreptunghi_inalt/descr_neg',descr_lung)







100%|██████████| 1000/1000 [01:01<00:00, 16.31it/s]


In [7]:
def descriptori_pozitivi(fisier_imagini,latime_fereastra, inaltime_fereastra, dim_celuala_hog, format ):
    files=os.listdir(fisier_imagini)
    descriptori_dad = []
    descriptori_deedee = []
    descriptori_dexter = []
    descriptori_mom = []
    descriptori_unkown = []
    characters=['dad','deedee','dexter','mom','unknown']
    for pers in characters:
        for file in tqdm(files):
            # print(file)
            if file.startswith(pers):
                img = cv.imread(fisier_imagini + file)
                img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
                img = np.array(img, dtype=np.uint8)
                patch = cv.resize(img, (latime_fereastra, inaltime_fereastra))
                descriptor = hog(patch, pixels_per_cell=(dim_celuala_hog, dim_celuala_hog), cells_per_block=(2, 2), feature_vector=True, block_norm='L2-Hys',orientations = orient)
                descriptor_flipedlr = hog(np.fliplr(patch), pixels_per_cell=(dim_celuala_hog, dim_celuala_hog), cells_per_block=(2, 2), feature_vector=True, block_norm='L2-Hys',orientations = orient)
                # descriptor_flipedud = hog(np.flipud(patch), pixels_per_cell=(dim_celuala_hog, dim_celuala_hog), cells_per_block=(2, 2), feature_vector=True, block_norm='L2-Hys')
                rows, cols = patch.shape
                M = cv.getRotationMatrix2D((cols / 2, rows / 2), 7, 1)
                rotated = cv.warpAffine(patch, M, (cols, rows))
                descriptor_rotated = hog(rotated, pixels_per_cell=(dim_celuala_hog, dim_celuala_hog), cells_per_block=(2, 2), feature_vector=True, block_norm='L2-Hys',orientations = orient)
                if pers == 'dad':
                    descriptori_dad.append(descriptor)
                    descriptori_dad.append(descriptor_flipedlr)
                    # descriptori_dad.append(descriptor_flipedud)
                    descriptori_dad.append(descriptor_rotated)
                if pers == 'deedee':
                    descriptori_deedee.append(descriptor)
                    descriptori_deedee.append(descriptor_flipedlr)
                    # descriptori_deedee.append(descriptor_flipedud)
                    descriptori_deedee.append(descriptor_rotated)
                if pers == 'dexter':
                    descriptori_dexter.append(descriptor)
                    descriptori_dexter.append(descriptor_flipedlr)
                    # descriptori_dexter.append(descriptor_flipedud)
                    descriptori_dexter.append(descriptor_rotated)
                if pers == 'mom':
                    descriptori_mom.append(descriptor)
                    descriptori_mom.append(descriptor_flipedlr)
                    # descriptori_mom.append(descriptor_flipedud)
                    descriptori_mom.append(descriptor_rotated)
                if pers == 'unknown':
                    descriptori_unkown.append(descriptor)
                    descriptori_unkown.append(descriptor_flipedlr)
                    # descriptori_unkown.append(descriptor_flipedud)
                    descriptori_unkown.append(descriptor_rotated)
    
    np.save(f'./descriptori_pozitivi/{format}/dad',descriptori_dad)

    np.save(f'./descriptori_pozitivi/{format}/deedee',descriptori_deedee)

    np.save(f'./descriptori_pozitivi/{format}/dexter',descriptori_dexter)

    np.save(f'./descriptori_pozitivi/{format}/mom',descriptori_mom)

    np.save(f'./descriptori_pozitivi/{format}/unknown',descriptori_unkown)
   

In [8]:
os.makedirs('./descriptori_pozitivi',exist_ok=True)
os.makedirs('./descriptori_pozitivi/patrat',exist_ok=True)
os.makedirs('./descriptori_pozitivi/dreptunghi',exist_ok=True)
os.makedirs('./descriptori_pozitivi/dreptunghi_inalt',exist_ok=True)

In [9]:
descriptori_pozitivi('./exemple_pozitive_nerestrictive/patrat/',dim_fereastra_patrata,dim_fereastra_patrata,dim_celuala_hog_patrat,'patrat')
descriptori_pozitivi('./exemple_pozitive_nerestrictive/dreptunghi/',latime_fereastra_dreptunghi,inaltime_fereastra_dreptunghi,dim_celula_hog_dreptunghi,'dreptunghi')
descriptori_pozitivi('./exemple_pozitive_nerestrictive/dreptunghi_inalt/',inaltime_fereastra_dreptunghi,latime_fereastra_dreptunghi,dim_celula_hog_dreptunghi,'dreptunghi_inalt')


100%|██████████| 2639/2639 [00:04<00:00, 630.98it/s]  
